In [3]:
from llama_index.core import ServiceContext, SimpleDirectoryReader, SummaryIndex
from llama_index.core.callbacks import CallbackManager
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from transformers import pipeline

In [ ]:
# Set context window size and number of output tokens
context_window = 2048
num_output = 256

# Store the pipeline/model outside of the LLM class to avoid memory issues
model_name = "ai-forever/mGPT"
pipeline = pipeline(
    "text-generation",
    model=model_name,
)

In [ ]:
from typing import Any
class OurLLM(CustomLLM):
    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=context_window,
            num_output=num_output,
            model_name=model_name,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        prompt_length = len(prompt)
        response = pipeline(prompt, max_new_tokens=num_output)[0][
            "generated_text"
        ]

        # Only return newly generated tokens
        text = response[prompt_length:]
        return CompletionResponse(text=text)

    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> CompletionResponseGen:
        raise NotImplementedError()

In [ ]:
# Define our LLM
llm = OurLLM()

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    context_window=context_window,
    num_output=num_output,
)

C:\Users\artem\AppData\Local\Temp\ipykernel_4328\3695658623.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(
g:\GovermentRecomendationProgs\ProgEnv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\artem\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\Local\llama_index\models\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingf

In [ ]:
from llama_index.core import VectorStoreIndex
# Load your data
documents = SimpleDirectoryReader("../data/").load_data()


g:\GovermentRecomendationProgs\ProgEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'SimpleDirectoryReader' is not defined

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)


In [ ]:
# Query and print response

query_engine = index.as_query_engine()
response = query_engine.query("What In directori")
print(response)

In [ ]:
index.storage_context.persist(persist_dir="../data/persisted_index")